Import the required ML libraries

In [1]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText
from pyspark.sql.functions import col
from pyspark.sql.utils import AnalysisException

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 3, Finished, Available)

In [2]:
df = spark.sql("SELECT * FROM twitterdata.final_tweets")

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 4, Finished, Available)

In [3]:
sentiment = (
    AnalyzeText()
    .setKind("SentimentAnalysis")
    .setTextCol("Tweet")
    .setOutputCol("response")
    .setErrorCol("error")
)

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 5, Finished, Available)

In [4]:
result = sentiment.transform(df)

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 6, Finished, Available)

In [5]:
display(result)

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6a17b8f9-7da1-4226-87bb-9490913b2564)

In [6]:
sent_df = result.withColumn('sentiment',col('response.documents.sentiment'))

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 8, Finished, Available)

In [7]:
display(sent_df)

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 488da6fd-3b9e-49fb-8d70-6935c508b05d)

In [8]:
df = sent_df.select('id', 'Created_At', 'Tweet', 'sentiment')

# Define the table name and temporary view name
table_name = "tweets_with_sentiments"
df.write.format('delta').mode('overwrite').saveAsTable(table_name)

StatementMeta(, 674fd057-ccae-41f8-8487-8cb4597f39b2, 10, Submitted, Running)

Table Already Exists


temp_view_name = "vw_df"

try:
    # Try to create the table
    df.write.format('delta').saveAsTable(table_name)
except AnalysisException:
    print('Table Already Exists')
   

    # Create or replace the temporary view
    df.createOrReplaceTempView(temp_view_name)

    # Perform the MERGE operation
    spark.sql(f"""
            MERGE INTO {table_name} target_table
            USING {temp_view_name} source_view
            ON source_view.id = target_table.id
            WHEN MATCHED AND
                 source_view.Created_At <> target_table.Created_At OR
                 source_view.Tweet <> target_table.Tweet
            THEN UPDATE SET *
            WHEN NOT MATCHED THEN INSERT *